작성자: 김승철
이메일: cheul1987@gmail.com

현 스크립트는 2021년 빅콘테스트 수산Biz 데이터를 활용하여 수산물 단가 예측을 하기위해 작성되었습니다. 오리지날 버젼은 seafood.py 파일로, 참가자 개인 PC로 진행되었으며, 구글 코랩의 경우 원활한 진행(상대적 저스펙)을 위해 코드를 간략화 하여 결과 보고서 내용이 아주 미세하게 일부 다를 수 있습니다.

# 데이터 활용 현황
- 트레이닝 세트 (Training set)
  - 2021빅콘테스트_데이터분석분야_챔피언리그_수산Biz_문제데이터
  - 트레이닝 세트의 20%를 검증 세트(Validation set)로 사용 (교차 검증 활용)
- 테스트 세트1 (Test set1)
  - 2021빅콘테스트_데이터분석분야_챔피언리그_수산Biz_자율평가데이터
  - 테스트 세트2 예측단가 예상 시, 추가 트레이닝 세트로 활용
- 테스트 세트2 (Test set2)
  - 2021 빅콘테스트_데이터분석분야_챔피언리그_수산Biz_평가데이터
  - 주요 특징(Feature)들이 생략되어 있어 예측단가 측정 차이 클 것으로 예상

## 테스트 세트1 분석 모델 수립
- 데이터: 트레이닝 세트와 테스트 세트 1을 활용
- 목표
  - 교차 검증(cross validation)을 이용한 모델 트레이닝
    - 검증 세트 예측단가와 검증 세트 실제 단가 차이 최소화
    - 단가 차이는 평균 절대 오차(Mean Absolute Error: MAE)를 활용
  - 트레이닝 된 모델에 테스트 세트1이 주어졌을 때 예측단가와 테스트 세트1의 실제 단가 차이로 측정되는 MAE를 최소화
  - 다양한 모델을 트레이닝하여 MAE 최소화 모델 발견
    - 회귀 (regression) 모델 필요
    - 모델 후보: XGBRegression, LinearRegression, Ridge, Lasso, ElasticNet
    - 모델 평가: cross validation
    - 하이퍼 변수(hyper parameters) 최적화: gridsearchCV

In [4]:
import os
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

### 데이터 전처리 (트레이닝 세트 & 테스트 세트1)
  - 단가(P_PRICE)가 제공되지 않은 행(row) 제외
  - 종속 변수(P_PRICE)와 독립변수(features) 분리
  - 날짜(REG_DATE) 데이터 타입과 포맷 변경
    - datetime64[ns] -> int
    - 월/일/년 -> 년월일
  - 상품 타입(P_TYPE)은 제거
    - 해당 행 값은 항상 ‘수산물‘이므로 데이터 처리에 불필요
  - 상품 수입 타입(P_IMPORT_TYPE)에 원 핫 엔코딩 적용
    - 셀(cell)은 여러 값을 포함하고 있으므로 값들을 분리하고, 고유값을 레이벨(label)로 새로운데이터 프레임 생성. 고유값 포함은 0,1로 표기.
    - 기존 데이터 프레임과 새로운 데이터 프레임을 합성 후 기존 상품 수입 타입은 삭제
    - 트레이닝 세트와 테스트 세트에 따라 새로이 형성된 데이터 프레임 형태 통일
  - 데이터 타입(dtype) 기반으로 열(columns) 분리
    - 수치 열(dtype = int/float) – REG_DATE, P_IMPORT_TYPE 원 핫 엔코딩 결과물
    - 카테고리 열(dtype = str) – CTRY_1, CTRY_2, … 등 수치 열을 제외한 모든 열
  - 모든 열의 경우 SimpleImputer를 이용, 미싱값(missing values) 0으로 처리
  - REG_DATE의 경우 표준화(StandardScaler) 적용
  - 모든 카테고리 열의 경우 원 핫 엔코딩 적용
    - 테스트동안 모르는 값 (unknown values) 발생 시 무시

In [5]:
# 데이터의 경우 직접 업로드 필요 (구글 코랩 좌측 파일 박스(Files) 연 후 복사 붙이기)
train_data = pd.read_excel('2021 빅콘테스트_데이터분석분야_챔피언리그_수산Biz_문제데이터.xlsx')
test_data = pd.read_excel('2021 빅콘테스트_데이터분석분야_챔피언리그_수산Biz_자율평가데이터.xlsx')

## P_PRICE 값이 없는 아이템(row) 제거
if train_data.P_PRICE.isnull().any():
    train_data.dropna(axis=0, subset=['P_PRICE'], inplace=True)
elif test_data.P_PRICE.isnull().any():
    test_data.dropna(axis=0, subset=['P_PRICE'], inplace=True)

## 독립변수와 종속변수(P_PRICE) 분리
y_train = train_data.P_PRICE 
X_train = train_data.drop(labels=['P_PRICE'], axis=1)

y_test = test_data.P_PRICE
X_test = test_data.drop(labels=['P_PRICE'], axis=1)

## REG_DATE 데이터 타입(datetime64[ns]) 변경 -> int 
X_train.REG_DATE = X_train.REG_DATE.apply(lambda x: int(x.strftime('%y%m%d')))
X_test.REG_DATE = X_test.REG_DATE.apply(lambda x: int(x.strftime('%y%m%d')))

## P_TYPE 제거: 값이 항상 수산물이기 때문에 데이터 처리에 불필요
X_train.drop(labels=['P_TYPE'], axis=1, inplace=True)
X_test.drop(labels=['P_TYPE'], axis=1, inplace=True)

## Manual One Hot Encoding 필요: P_IMPORT_TYPE 은 여러 요소의 값으로 이루어졌으므로 고유값 분리 후 적용
## 기존 데이터 프레임과 새로운 데이터 프레임을 합친 후, 상품 수입 타입(P_IMPORT_TYPE)은 제거
def converter(column):
    '''주어진 Column에 쉼표(,)를 기준으로 셀(cell)값을 분리 후 고유값을 레이벨(Labels)로 하는 데이터프레임 생성. 
    고유값 포함은 0,1로 표기.'''
    temp = column.copy()
    temp = temp.str.split(',')
    temp = temp.explode()
    temp = pd.crosstab(temp.index, temp)
    return temp

X_train_P_IMPORT_TYPE = converter(X_train.P_IMPORT_TYPE)
X_test_P_IMPORT_TYPE = converter(X_test.P_IMPORT_TYPE)
X_train.drop(labels=['P_IMPORT_TYPE'], axis=1, inplace=True)
X_test.drop(labels=['P_IMPORT_TYPE'], axis=1, inplace=True)
X_train = X_train.join(X_train_P_IMPORT_TYPE)
X_test = X_test.join(X_test_P_IMPORT_TYPE)

## 트레이닝 세트와 테스트 세트에 따라 새로이 형성된 데이터 프레임 형태 통일
feature_diff = set(X_train_P_IMPORT_TYPE) - set(X_test_P_IMPORT_TYPE)
feature_diff_df = pd.DataFrame(data=np.zeros((X_test_P_IMPORT_TYPE.shape[0], len(feature_diff))),
                               columns=list(feature_diff), dtype=int)

X_test = X_test.join(feature_diff_df)
X_test = X_test[list(X_train.columns)]

## 열(columns) 분리 (데이터 타입(dtype) 기반) 후 해당 트랜스포머 파이프라인에 저장
numerical_cols = ['REG_DATE']
numerical_cols_oh = [col for col in X_train.columns if X_train[col].dtype in ['int64', 'float64']]
numerical_cols_oh.remove('REG_DATE')
categorical_cols = [col for col in X_train.columns if X_train[col].dtype == 'object']

numerical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant')),
                                        ('scale', StandardScaler())])

numerical_transformer_oh = SimpleImputer(strategy='constant')

categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant')),
                                          ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(transformers=[('num', numerical_transformer, numerical_cols),
                                               ('num_oh', numerical_transformer_oh, numerical_cols_oh),
                                               ('cat', categorical_transformer, categorical_cols)])

### 모델 적용 및 해달 모델 하이퍼 변수(hyper-parameters) 최적화
  - XGBRegression 모델의 경우, 트레이닝 세트를 이용한 교차검증(Cross-validation)과 테스트 세트 예측에서 가장 낮은 평균 절대 오차를 보입니다.
  - XGBRegression의 경우 Gradient Boosting을 활용하는 알고리즘이며, 이는
판단 트리 (Decision tree)와 같은 위크 러너(weak learner)의 단점을 보완합니다. 따라서 사용된 다른 알고리듬에 비해서 상대적으로 뛰어난 결과를 보여주는걸로 예상됩니다.

In [6]:
models = [XGBRegressor(random_state=0), LinearRegression(), Ridge(random_state=0), Lasso(random_state=0), ElasticNet(random_state=0)]
params = [{'clf__n_estimators': [800, 1000],'clf__max_depth': [6,8], 'clf__learning_rate': [0.1]},
          {},
          {'clf__alpha': [0.1, 1.0]},
          {'clf__alpha': [0.1, 1.0]},
          {'clf__alpha': [0.1, 1.0], 'clf__l1_ratio': [0.5, 0.8]}]
for mod, par in zip(models, params):
    X_valid, y_valid = X_train.copy(), y_train.copy()
    my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                  ('clf', mod)])
    clf = GridSearchCV(estimator = my_pipeline,
                        param_grid = par,
                        scoring='neg_mean_absolute_error',
                        cv=5)
    clf.fit(X_valid, y_valid)
    pred = clf.best_estimator_.predict(X_test)
    print(f'\nEstimator: {mod} \nHyperparams: {clf.best_params_}, MAE(training): {-1 * clf.best_score_:.3f}, MAE(test): {mean_absolute_error(pred, y_test):.3f}\n')
    print('Product  \t\t\tPredict\t\t\tReal')
    for i in range(10):
        print("{:7}\t\t\t{:.2f}\t\t\t{:.2f}".format(X_test.P_NAME.iloc[i], pred[i], y_test[i]))
    print('\n')

[07:54:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:55:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:55:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:55:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:55:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:56:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:56:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:56:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

### 최적화 XGBRegression 및 주요 오류 상품 조사
  - 테스트 세트 평균 절대 오차를 조사하였을 때 특정 아이템의 편차가 큼을 발견
    - 테스트 MAE > 20 시행: 123 아이템 발견(성게알 85, 해삼 15)
    - 성게알 제외 후 -> 트레이닝 MAE : 1.43, 테스트 MAE : 1.312
    - 성게알과 해삼 제외 후 -> 트레이닝 MAE : 1.310, 테스트 MAE : 1.25
  - 성게알, 해삼 관련 데이터가 보완 될 경우 더 좋은 모델로 성장할 것으로 예상    

In [7]:
new_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('xgb', XGBRegressor(random_state=0))])
xgb = GridSearchCV(estimator = new_pipeline,
                    param_grid = {'xgb__n_estimators': [1000],
                                  'xgb__max_depth': [8], 
                                  'xgb__learning_rate': [0.1]},
                    scoring='neg_mean_absolute_error',
                    n_jobs=-1,
                    cv=5)
xgb.fit(X_train, y_train)
pred = xgb.best_estimator_.predict(X_test)
count = 0
print('주 오류 상품 조사')
print('Product  \t\t\tPredict\t\t\tReal')
for i in range(y_test.shape[0]):
    if abs(pred[i] - y_test[i]) > 20.0:
        print("{:7}\t\t\t{:.2f}\t\t\t{:.2f}".format(X_test.P_NAME.iloc[i], pred[i], y_test[i]))
        count = count + 1
print(f'{count}항목에서 MAE>20.0 발견')

[08:03:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
주 오류 상품 조사
Product  			Predict			Real
성게알    			336.91			362.06
성게알    			18.96			80.00
성게알    			93.32			194.50
성게알    			336.91			316.15
성게알    			93.32			120.63
해삼     			92.26			53.00
성게알    			336.91			220.96
성게알    			336.91			242.61
성게알    			69.26			44.87
성게알    			336.91			262.52
해삼     			84.62			106.56
성게알    			336.91			260.56
성게알    			336.91			255.46
낙지     			11.17			36.42
성게알    			336.91			205.14
왕게     			36.99			63.50
낙지     			11.17			36.42
해삼     			92.26			62.41
청새리상어  			156.82			229.57
해삼     			191.52			63.50
흰다리새우  			11.65			34.16
성게알    			336.91			233.95
성게알    			336.91			307.84
낙지     			11.17			31.22
성게알    			93.32			128.36
성게알    			336.91			269.58
해삼     			240.42			265.00
성게알    			336.91			242.49
성게알    			93.32			141.74
성게알    			336.91			236.17
성게알    			336.91			194.45
성게알    			336.91			193.62
성게알    			336.91			207.63
성

### 주요 오류 상품인 성개알 제거 후 트레이닝 & 테스트

In [8]:
## 성개알 제거 후 트레이닝 및 테스트 
inx = []
for i in range(X_train.shape[0]):
    if X_train.P_NAME.iloc[i] == '성게알':
        inx.append(i)
X_train_s = X_train.drop(index = inx)
y_train_s = y_train.drop(index = inx)
X_train_s.reset_index(drop=True, inplace=True)
y_train_s.reset_index(drop=True, inplace=True)

inx = []
for i in range(y_test.shape[0]):
    if X_test.P_NAME.iloc[i] == '성게알':
        inx.append(i)
X_test_s = X_test.drop(index = inx)
y_test_s = y_test.drop(index = inx)
X_test_s.reset_index(drop=True, inplace=True)
y_test_s.reset_index(drop=True, inplace=True)

xgb.fit(X_train_s, y_train_s)
pred = xgb.best_estimator_.predict(X_test_s)
print(f'\n성게알 제거 후 MAE(training): {-1 * xgb.best_score_:.3f}, MAE(test): {mean_absolute_error(pred, y_test_s):.3f}')

[08:07:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.

성게알 제거 후 MAE(training): 1.430, MAE(test): 1.312


### 주요 오류 상품인 성개알과 해삼 제거 후 트레이닝 & 테스트

In [9]:
## 성게알과 해삼 제거 후 트레이밍 및 테스트
inx = []
for i in range(X_train.shape[0]):
    if X_train.P_NAME.iloc[i] == '성게알' or X_train.P_NAME.iloc[i] == '해삼':
        inx.append(i)
X_train_sh = X_train.drop(index = inx)
y_train_sh = y_train.drop(index = inx)
X_train_sh.reset_index(drop=True, inplace=True)
y_train_sh.reset_index(drop=True, inplace=True)

inx = []
for i in range(y_test.shape[0]):
    if X_test.P_NAME.iloc[i] == '성게알' or X_test.P_NAME.iloc[i] == '해삼':
        inx.append(i)
X_test_sh = X_test.drop(index = inx)
y_test_sh = y_test.drop(index = inx)
X_test_sh.reset_index(drop=True, inplace=True)
y_test_sh.reset_index(drop=True, inplace=True)

xgb.fit(X_train_sh, y_train_sh)
pred = xgb.best_estimator_.predict(X_test_sh)
print(f'\n성게알과 해삼 제거 후 MAE(training): {-1 * xgb.best_score_:.3f}, MAE(test): {mean_absolute_error(pred, y_test_sh):.3f}\n')

[08:10:40] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.

성게알과 해삼 제거 후 MAE(training): 1.310, MAE(test): 1.250



## 테스트 세트2 예측 모델 수립
  - 데이터: 트레이닝 세트와 테스트 세트 1&2를 활용
  - 목표
    - 새로운 트레이닝 데이터 생성: 기존 트레이닝 세트 + 테스트 세트1
    - 교차 검증(cross validation)을 이용한 모델 트레이닝
      - 검증 세트 예측단가와 검증 세트 실제 단가 차이 최소화
      - 값의 차이는 평균 절대 오차(Mean Absolute Error)를 이용
    - 트레이닝 된 모델에 테스트 세트2가 주어졌을 때 단가 예측
    - 다양한 모델을 트레이닝하여 교차 검증 MAE 최소화 모델 발견
      - 회귀 (regression) 모델 필요
      - 모델 후보: XGBRegression, LinearRegression, Ridge, Lasso, ElasticNet
      - 모델 평가: cross validation
      - 하이퍼 변수(hyper parameters) 최적화: gridsearchCV

### 데이터 전처리(테스트 세트2)
  - 테스트 세트2
    - 불필요한 인덱스 제거 (행[0])
    - NaN 처리: 어종(CATEGORY_2), 상세어종(P_NAME)
    - 어종(CATEGORY_2), 상세어종(P_NAME), 일자(REG_DATE), 예측단가(P_PRICE) 레이벨링
    - 종속 변수(P_PRICE)와 독립변수(features) 분리
    - 날짜(REG_DATE) 데이터 타입과 포맷 변경
      - datetime64[ns] -> int
      - 월/일/년 -> 년월일
  - 기존 트레이닝 세트와 테스트 세트1의 선택된 특정 독립 변수(어종, 상세어종, 일자)와 종속 변수(단가)에 해당하는 데이터만을 추출 후 새로운 트레이닝 세트 생성
  - 신규 트레이닝 세트 + 테스트 세트2
    - 데이터 타입(dtype) 기반으로 열(columns) 분리
      - 수치 열(dtype = int/float) – REG_DATE
      - 카테고리 열(dtype = str) – CATEGORY_2, P_NAME
    - 모든 열의 경우 SimpleImputer를 이용, 미싱값(missing values) 처리
    - REG_DATE의 경우 표준화(StandardScaler) 적용
    - 모든 카테고리 열의 경우 원 핫 엔코딩 적용
      - 테스트동안 모르는 값 (unknown values) 발생 시 무시

In [10]:
## 테스트 세트2 불러오기
test_data2 = pd.read_excel('2021 빅콘테스트_데이터분석분야_챔피언리그_수산Biz_평가데이터_update_210831.xlsx', 
                            header=None)

## 불필요한 인덱스 제거
test_data2.drop(index=0, inplace=True)
test_data2.reset_index(drop=True, inplace=True)

## NaN 채워넣기: 어종(CATEGORY_2), 상세어종(P_NAME)
missing_index = [0,1,5,6,10,11]
for i in missing_index:
    test_data2.iloc[:,i].fillna(test_data2.iloc[0,i], inplace=True)


## 데이터 처리 효율 향상을 위해 어종(CATEGORY_2), 상세어종(P_NAME), 일자(REG_DATE), 
## 예측 단가(P_PRICE)에 따라 데이터프레임 3 등분
columns = ['CATEGORY_2','P_NAME','REG_DATE','P_PRICE']
test_data2_1 = test_data2.iloc[:,:4]
test_data2_1.columns = columns
test_data2_2 = test_data2.iloc[:,5:9]
test_data2_2.columns = columns
test_data2_3 = test_data2.iloc[:,10:14]
test_data2_3.columns = columns


## 독립변수와 종속변수(P_PRICE) 분리
X_test2_1 = test_data2_1.drop(labels='P_PRICE', axis=1)
X_test2_2 = test_data2_2.drop(labels='P_PRICE', axis=1)
X_test2_3 = test_data2_3.drop(labels='P_PRICE', axis=1)


## REG_DATE 데이터 타입(datetime64[ns]) 변경 -> int 
X_test2_1.REG_DATE = X_test2_1.REG_DATE.apply(lambda x: int(x.strftime('%y%m%d')))
X_test2_2.REG_DATE = X_test2_2.REG_DATE.apply(lambda x: int(x.strftime('%y%m%d')))
X_test2_3.REG_DATE = X_test2_3.REG_DATE.apply(lambda x: int(x.strftime('%y%m%d')))


## 트레이닝 세트와 테스트 세트1의 특정 독립 변수(어종, 상세어종, 날짜, 가격)와 
## 종속 변수(가격)에 해당하는 데이터를 추출 후 새로운 트레이닝 세트 작성
y_train2 = y_train.copy()
X_train2 = X_train.loc[:,['CATEGORY_2','P_NAME','REG_DATE']].copy()
y_test2 = y_test.copy()
X_test2 = X_test.loc[:,['CATEGORY_2','P_NAME','REG_DATE']].copy()

y_train2 = y_train2.append(y_test2, ignore_index=True)
X_train2 = X_train2.append(X_test2, ignore_index=True)


## 열(columns) 분리 (데이터 타입(dtype) 기반) 후 해당 트랜스포머 파이프라인에 저장
numeric_cols = ['REG_DATE']
categoric_cols = ['CATEGORY_2','P_NAME']

numeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant')),
                                      ('scale', StandardScaler())])

categoric_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant')),
                                        ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocess = ColumnTransformer(transformers=[('num', numeric_transformer, numeric_cols),
                                             ('cat', categoric_transformer, categoric_cols)])

### 모델 적용 및 해달 모델 하이퍼 변수(hyper-parameters) 최적화

In [11]:
models = [XGBRegressor(random_state=0), LinearRegression(), Ridge(random_state=0), Lasso(random_state=0), ElasticNet(random_state=0)]
params = [{'clf__n_estimators': [800, 1000],'clf__max_depth': [6,8], 'clf__learning_rate': [0.1, 0.5]},
          {},
          {'clf__alpha': [0.1, 1.0]},
          {'clf__alpha': [0.1, 1.0]},
          {'clf__alpha': [0.1, 1.0], 'clf__l1_ratio': [0.5, 0.8]}]
for mod, par in zip(models, params):
    X_valid, y_valid = X_train2.copy(), y_train2.copy()
    my_pipeline = Pipeline(steps=[('preprocessor', preprocess),
                                  ('clf', mod)])
    clf = GridSearchCV(estimator = my_pipeline,
                        param_grid = par,
                        scoring='neg_mean_absolute_error',
                        n_jobs=-1,
                        cv=5)
    clf.fit(X_valid, y_valid)
    print(f'\nEstimator: {mod} \nHyperparams: {clf.best_params_}, MAE(training): {-1 * clf.best_score_:.3f}\n')

[08:19:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.

Estimator: XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1) 
Hyperparams: {'clf__learning_rate': 0.1, 'clf__max_depth': 6, 'clf__n_estimators': 1000}, MAE(training): 3.680


Estimator: LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False) 
Hyperparams: {}, MAE(training): 3.822


Estimator: Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=0, solver='auto', tol

### 최적화 XGBRegression과 LinearRegression 사용 및 데이터 출력
- XGBRegression의 경우 Gradient Boosting 알고리듬을 활용하기에 큰틀을 잡고 측정하는데는 뛰어난 장점이 있지만, 그 장점 때문에 아주 미세한 차이를 놓치게 되어 LinearRegression 보다 뒤쳐지는 것으로 보입니다. 

In [16]:
# XGBRegression
new_pipeline = Pipeline(steps=[('preprocessor', preprocess),
                              ('xgb', XGBRegressor(random_state=0))])
xgb = GridSearchCV(estimator = new_pipeline,
                   param_grid = {'xgb__n_estimators': [1000],
                                 'xgb__max_depth': [6], 
                                 'xgb__learning_rate': [0.1]},
                   scoring='neg_mean_absolute_error',
                   n_jobs=-1,
                   cv=5)
X_valid, y_valid = X_train2.copy(), y_train2.copy()
xgb.fit(X_valid, y_valid)
print('XGBRegressor')
for s in [X_test2_1, X_test2_2, X_test2_3]:
    name = s.P_NAME.iloc[0]
    pred = xgb.best_estimator_.predict(s)
    print(f'{name}\n{pred}')
    
X_test2_cum = (X_test2_1.append(X_test2_2, ignore_index=True)).append(X_test2_3, ignore_index=True)    
pred = xgb.predict(X_test2_cum)
X_test2_cum['P_PRICE'] = pred
X_test2_cum.REG_DATE = pd.to_datetime(X_test2_cum.REG_DATE, format='%y%m%d').dt.date
X_test2_cum.columns = ['품목', '상세품목', '날짜', '예측단가']
X_test2_cum.to_excel('2021 빅콘테스트_데이터분석분야_챔피언리그_수산Biz_평가데이터_update_210831_예측(xgb).xlsx',
                     index=False)

# LinearRegression
new_pipeline = Pipeline(steps=[('preprocessor', preprocess),
                              ('lR', LinearRegression())])
linR = GridSearchCV(estimator = new_pipeline,
                    param_grid = {},
                    scoring='neg_mean_absolute_error',
                    n_jobs=-1,
                    cv=5)
X_valid, y_valid = X_train2.copy(), y_train2.copy()
linR.fit(X_valid, y_valid)
print('\nLinearRegression')
for s in [X_test2_1, X_test2_2, X_test2_3]:
    name = s.P_NAME.iloc[0]
    pred = linR.best_estimator_.predict(s)
    print(f'{name}\n{pred}')
    
X_test2_cum = (X_test2_1.append(X_test2_2, ignore_index=True)).append(X_test2_3, ignore_index=True)    
pred = linR.predict(X_test2_cum)
X_test2_cum['P_PRICE'] = pred
X_test2_cum.REG_DATE = pd.to_datetime(X_test2_cum.REG_DATE, format='%y%m%d').dt.date
X_test2_cum.columns = ['품목', '상세품목', '날짜', '예측단가']
X_test2_cum.to_excel('2021 빅콘테스트_데이터분석분야_챔피언리그_수산Biz_평가데이터_update_210831_예측(LR).xlsx',
                     index=False)

[08:36:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGBRegressor
오징어
[2.8268774 2.8268774 2.8268774 2.8268774 2.8268774 2.8268774 2.8268774
 2.8268774 2.8268774 2.8268774 2.8268774 2.8268774 2.8268774 2.8268774
 2.8268774 2.8268774 2.8268774 2.8268774 2.8268774 2.8268774 2.8268774
 2.8268774 2.8268774 2.8268774 2.8268774 2.8268774]
연어
[13.332984 13.332984 13.332984 13.332984 13.332984 13.332984 13.332984
 13.332984 13.332984 13.332984 13.332984 13.332984 13.332984 13.332984
 13.332984 13.332984 13.332984 13.332984 13.332984 13.332984 13.332984
 13.332984 13.332984 13.332984 13.332984 13.332984]
흰다리새우
[10.0239525 10.0239525 10.0239525 10.0239525 10.0239525 10.0239525
 10.0239525 10.0239525 10.0239525 10.0239525 10.0239525 10.0239525
 10.0239525 10.0239525 10.0239525 10.0239525 10.0239525 10.0239525
 10.0239525 10.0239525 10.0239525 10.0239525 10.0239525 10.0239525
 10.0239525 10.0239525]

LinearRegression
오징어
[3.

## 트레이닝 데이터를 테스트 데이터에 편향하여 조작한 경우
  - 트레이닝 세트에서 오징어, 연어, 흰다리 새우를 제외한 나머지 데이터들을 제거 하고 트레이닝 할 경우, 일자의 비중이 올라가기 때문에 테스트 결과가 훨씬 정밀하게 나타날 수 있습니다. 하지만 테스트 세트의 데이터를 미리 보고 거기에 맞추어 모델 트레이닝을 하는건 실제론 불가능하지만 차이 비교를 위해 진행했습니다.
  - 오징어, 연어, 흰다리 새우를 제외한 모든행 제거후 트레이닝 및 테스트 결과

In [14]:
inx = []
for i in range(X_train2.shape[0]):
    if X_train2.P_NAME.iloc[i] == '오징어' or X_train2.P_NAME.iloc[i] == '연어' or X_train2.P_NAME.iloc[i] == '흰다리새우':
        inx.append(i)
X_train2_chosen = X_train2.iloc[inx]
y_train2_chosen = y_train2.iloc[inx]
X_train2_chosen.reset_index(drop=True, inplace=True)
y_train2_chosen.reset_index(drop=True, inplace=True)

models = [XGBRegressor(random_state=0), LinearRegression(), Ridge(random_state=0), Lasso(random_state=0), ElasticNet(random_state=0)]
params = [{'clf__n_estimators': [800, 1000],'clf__max_depth': [6,8], 'clf__learning_rate': [0.1, 0.5]},
          {},
          {'clf__alpha': [0.1, 1.0]},
          {'clf__alpha': [0.1, 1.0]},
          {'clf__alpha': [0.1, 1.0], 'clf__l1_ratio': [0.5, 0.8]}]
for mod, par in zip(models, params):
    X_valid, y_valid = X_train2_chosen.copy(), y_train2_chosen.copy()
    my_pipeline = Pipeline(steps=[('preprocessor', preprocess),
                                  ('clf', mod)])
    clf = GridSearchCV(estimator = my_pipeline,
                       param_grid = par,
                       scoring='neg_mean_absolute_error',
                       n_jobs=-1,
                       cv=5)
    clf.fit(X_valid, y_valid)
    print(f'\nEstimator: {mod} \nHyperparams: {clf.best_params_}, MAE(training): {-1 * clf.best_score_:.3f}\n')

## 최적화 LinearRegression 사용 및 데이터 출력
new_pipeline = Pipeline(steps=[('preprocessor', preprocess),
                              ('lR', LinearRegression())])
linR = GridSearchCV(estimator = new_pipeline,
                    param_grid = {},
                    scoring='neg_mean_absolute_error',
                    n_jobs=-1,
                    cv=5)
X_valid, y_valid = X_train2_chosen.copy(), y_train2_chosen.copy()
linR.fit(X_valid, y_valid)
print('\nLinearRegression')
for s in [X_test2_1, X_test2_2, X_test2_3]:
    name = s.P_NAME.iloc[0]
    pred = linR.best_estimator_.predict(s)
    print(f'{name}\n{pred}')

X_test2_cum = (X_test2_1.append(X_test2_2, ignore_index=True)).append(X_test2_3, ignore_index=True)    
pred = linR.predict(X_test2_cum)
X_test2_cum['P_PRICE'] = pred
X_test2_cum.REG_DATE = pd.to_datetime(X_test2_cum.REG_DATE, format='%y%m%d').dt.date
X_test2_cum.columns = ['품목', '상세품목', '날짜', '예측단가']
X_test2_cum.to_excel('2021 빅콘테스트_데이터분석분야_챔피언리그_수산Biz_평가데이터_update_210831_예측(LR)_제거.xlsx',
                     index=False)

[08:28:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.

Estimator: XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1) 
Hyperparams: {'clf__learning_rate': 0.1, 'clf__max_depth': 8, 'clf__n_estimators': 800}, MAE(training): 2.877


Estimator: LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False) 
Hyperparams: {}, MAE(training): 2.886


Estimator: Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=0, solver='auto', tol=